This is a benchmark for Autogluon Forecasting task.  
Assume that we have two COV19 dataset, train_data.csv and test_data.csv in the following format:  

|Date|ConfirmedCases|name|
| ------ | ------ | ------ |
|2020-01-22|0.0|Afghanistan_|
|2020-01-23|0.0|Afghanistan_|
|2020-01-24|0.0|Afghanistan_|
|2020-01-25|0.0|Afghanistan_|
|2020-01-26|0.0|Afghanistan_|

The comfirmedcases here is the cummulative comfirmed cases up to that date in a certain country.

In [ ]:
# Load dataset
from autogluon.forecasting.task.forecasting.forecasting import Forecasting as task

train_data = task.Dataset("https://autogluon.s3-us-west-2.amazonaws.com/datasets/CovidTimeSeries/train.csv")
test_data = task.Dataset("https://autogluon.s3-us-west-2.amazonaws.com/datasets/CovidTimeSeries/test.csv")
prediction_length = 19
eval_metric = "mean_wQuantileLoss"

In [ ]:
train_data.head()

## Directly using GluonTS to do forecasting

In [ ]:
import pandas as pd


def rebuild_tabular(X, time_column, target_column, index_column=None):
    if index_column is None:
        X = X[[time_column, target_column]]
        X["index_column"] = ["time_series" for i in range(X.shape[0])]
        index_column = "index_column"
    time_list = sorted(list(set(X[time_column])))
    freq = pd.infer_freq(time_list)
    if freq is None:
        raise ValueError("Freq cannot be inferred. Check your dataset.")

    def reshape_dataframe(df):
        df = df.sort_values(by=index_column)
        data_dic = {index_column: sorted(list(set(df[index_column])))}

        for time in time_list:
            tmp = df[df[time_column] == time][[index_column, time_column, target_column]]
            tmp = tmp.pivot(index=index_column, columns=time_column, values=target_column)
            tmp_values = tmp[time].values
            data_dic[time] = tmp_values
        return pd.DataFrame(data_dic)

    X = reshape_dataframe(X)
    return X

In [ ]:
rebuilt_train = rebuild_tabular(train_data, time_column="Date", target_column="ConfirmedCases", index_column="name")
rebuilt_test = rebuild_tabular(test_data, time_column="Date", target_column="ConfirmedCases", index_column="name")

In [ ]:
rebuilt_train.head()

In [ ]:
rebuilt_test.head()

Train-Val Split

In [ ]:
rebuilt_val = rebuilt_train.copy()
rebuilt_train = rebuilt_train.iloc[:, :-prediction_length]

Then transform the rebuilt tabular into GluonTS Listdata

In [ ]:
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset

def create_gluonts_data(df, index_column):
    index = df[index_column]
    target = df.drop(index_column, axis=1)
    target_values = target.values
    date_list = target.columns
    freq = pd.infer_freq(date_list)
    data = [
        {
            FieldName.TARGET: target,
            FieldName.START: pd.Timestamp(date_list[0], freq=freq),
            FieldName.ITEM_ID: item_id
        }
        for (target, item_id) in zip(target_values, index)
    ]
    return ListDataset(data, freq)

In [ ]:
gluonts_train_data = create_gluonts_data(rebuilt_train, "name")
gluonts_val_data = create_gluonts_data(rebuilt_val, "name")
gluonts_test_data = create_gluonts_data(rebuilt_test, "name")

Train and evaluate

In [ ]:
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.trainer import Trainer

params = {
    "prediction_length": 19,
    "freq": "D",
    "num_batches_per_epoch": 10,
    "epochs": 50,
    "quantiles": [0.1, 0.5, 0.9]
}
model = MQCNNEstimator.from_hyperparameters(**params)

predictor = model.train(gluonts_train_data)

In [ ]:
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from tqdm import tqdm


forecast_it, ts_it = make_evaluation_predictions(dataset=gluonts_test_data,
                                                 predictor=predictor,
                                                 num_samples=100)
forecasts, tss = list(forecast_it), list(ts_it)
# forecasts, tss = list(tqdm(forecast_it, total=len(gluonts_val_data))), list(tqdm(ts_it, total=len(gluonts_val_data)))
# print(forecasts[0], tss[0])
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(tss))
print(agg_metrics[eval_metric])

Get prediction results for 20 days after the dataset

In [ ]:
result_dict = {}
predicted_targets = list(predictor.predict(gluonts_test_data))
index = sorted(list(set(train_data["name"])))

quantiles = [0.1, 0.5, 0.9]
for i in range(len(index)):
    tmp_dict = {}
    for quantile in quantiles:
        tmp_dict[quantile] = predicted_targets[i].quantile(str(quantile))
    df = pd.DataFrame(tmp_dict)
    df.index = pd.date_range(start=predicted_targets[i].start_date,
                             periods=prediction_length,
                             freq="D")
    result_dict[index[i]] = df
    
print(result_dict["Afghanistan_"])

## Using AutoGluon To do this forecasting

In [ ]:
from autogluon.forecasting.task.forecasting.forecasting import Forecasting as task

import autogluon.core as ag

# change this to specify search strategy, can try bayesopt, random, or skopt
searcher_type = "random"
# change this to specify eval metric, one of ["MASE", "MAPE", "sMAPE", "mean_wQuantileLoss"]
eval_metric = "mean_wQuantileLoss"

predictor = task.fit(train_data=train_data,
                     prediction_length=19,
                     index_column="name",
                     target_column="ConfirmedCases",
                     time_column="Date",
                     hyperparameter_tune=True,
                     hyperparameters={"MQCNN": {'context_length': ag.Int(10, 20),
                                                'epochs': 50,
                                                "num_batches_per_epoch": 10}},
                     search_strategy=searcher_type,
                     eval_metric=eval_metric,
                     quantiles=[0.1, 0.5, 0.9],
                     num_trials=10)

Evaluate the model, and 

In [ ]:
print(predictor.evaluate(test_data))

Get prediction results for 20 days after the dataset

In [ ]:
predictions = predictor.predict(test_data, quantiles=[0.1, 0.5, 0.9])
print(predictions['Afghanistan_'])